# How to Prepare a Permutation Symmetric Multiqubit State on an Actual Quantum Computer


I've made a big deal over the fact that one can represent a spin-$j$ state equivalently as a permutation symmetric state of $2j$ qubits, not least because given that our quantum computers work with qubits, we need a way to represent everything we might care about in terms of them.

For example, if we have a spin-$\frac{3}{2}$ state, we could express it in the usual $\mid j, m \rangle$ basis as:

$ a\mid \frac{3}{2}, \frac{3}{2}\rangle + b\mid \frac{3}{2}, \frac{1}{2}\rangle + c\mid \frac{3}{2}, -\frac{1}{2}\rangle + d\mid \frac{3}{2}, -\frac{3}{2}\rangle\rangle $

or in terms of symmeterized qubits:

$ a\mid \uparrow \uparrow \uparrow \rangle + b\frac{1}{\sqrt{3}}(\mid \uparrow \uparrow \downarrow \rangle + \mid \uparrow \downarrow \uparrow \rangle + \mid \downarrow \uparrow \uparrow \rangle) + c\frac{1}{\sqrt{3}}(\mid \downarrow \downarrow \uparrow \rangle + \mid \downarrow \uparrow \downarrow \rangle + \mid \uparrow \downarrow \downarrow \rangle) + d\mid \downarrow \downarrow \downarrow \rangle$

In other words, there's a one-to-one correspondence between the four $\mid j, m \rangle$ basis states and the four symmetric basis states of three qubits. To wit: the state with $3 \uparrow$, with $2 \uparrow, 1 \downarrow$, with $2 \downarrow, 1 \uparrow$, and with $3 \downarrow$.

So we can easily form a linear map that takes us from the one representation to the other:

In [1]:
import numpy as np
import qutip as qt
from itertools import product

def spin_sym(j):
    n = int(2*j)
    if n == 0:
        return qt.Qobj([1])
    N = {}
    for p in product([0,1], repeat=n):
        if p.count(1) in N:
            N[p.count(1)] += qt.tensor(*[qt.basis(2, i) for i in p])
        else:
            N[p.count(1)] = qt.tensor(*[qt.basis(2, i) for i in p])
    Q = qt.Qobj(np.array([N[i].unit().full().T[0].tolist() for i in range(n+1)]))
    Q.dims[1] = [2]*n
    return Q.dag()

j = 3/2
S = spin_sym(j)

spin = qt.rand_ket(int(2*j+1))
sym = S*spin

print("spin state:\n%s" % spin)
print("\npermutation symmetric qubits:\n%s" % sym)

I = S.dag()*S
print("\ntransformation an isometry?: %s" % (I == qt.identity(I.shape[0])))

spin state:
Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[-0.33949479-0.39663995j]
 [-0.44666802+0.2061572j ]
 [-0.24466995-0.25950522j]
 [-0.40608237-0.43965633j]]

permutation symmetric qubits:
Quantum object: dims = [[2, 2, 2], [1]], shape = (8, 1), type = ket
Qobj data =
[[-0.33949479-0.39663995j]
 [-0.2578839 +0.11902491j]
 [-0.2578839 +0.11902491j]
 [-0.14126026-0.14982541j]
 [-0.2578839 +0.11902491j]
 [-0.14126026-0.14982541j]
 [-0.14126026-0.14982541j]
 [-0.40608237-0.43965633j]]

transformation an isometry?: True


<hr>

Another way of obtaining the same result is to find the roots of the Majorana polynomial of the spin, convert each into a qubit, and then symmeterize the qubits. Well, technically, if we do this, we lose the overall phase information of the original spin, but such is life.

Below we take a spin, finds its stars, convert them into qubits, symmeterize them, then use the inverse of the transformation above to return us to a spin state, and find the stars again. We see we recover the original stars exactly. So we get the right result up to phase.

In [2]:
from examples.magic import *
from itertools import permutations

def symmetrize(kets):
    return sum([qt.tensor(*[kets[p_] for p_ in p])\
                for p in permutations(list(range(len(kets))))]).unit()

stars = spin_XYZ(spin)
qubits = [qt.Qobj(xyz_spinor(star)) for star in stars]
sym2 = symmetrize(qubits)
spin2 = S.dag()*sym2
stars2 = spin_XYZ(spin2)

print("initial stars:\n%s\n" % "\n".join([str(star) for star in stars]))
print("final stars:\n%s\n" % "\n".join([str(star) for star in stars2]))

<IPython.core.display.Javascript object>

initial stars:
[ 0.04646562 -0.77615288 -0.62883038]
[-0.32173743  0.93071233  0.17395284]
[ 0.83174711 -0.3818807   0.40294401]

final stars:
[ 0.04646562 -0.77615288 -0.62883038]
[-0.32173743  0.93071233  0.17395284]
[ 0.83174711 -0.3818807   0.40294401]



<hr>

One way of saying this is that if we have a state of $2j$ separable qubits, each with an $(x, y, z)$ point representing its rotation axis, if we permute the qubits in all possible orders and add up all these permuted states, then the resulting state is naturally permutation symmetric, but remarkably preserves the $(x, y, z)$ points of each of the qubits. Each point is now encoded not individually in the separable qubits, but instead holistically in the state of the entangled whole. If we transform from the symmeterized qubits to the spin-$j$ state and find the roots of the Majorana polynomial, these complex roots, stereographically projected to the sphere, gives us back our original $(x, y, z)$ points. Of course, we have thrown out some information: namely, the phases of the individual qubits we symmeterized.

That's all well and good, but a moment's reflection will convince you that the operation of symmeterizing a bunch of qubits is *not* a unitary operation. After all, I need to make a separate copy of the qubits, one for each possible permutation, and then add them all up. This of course would violate no-cloning. 

So you might wonder: Suppose I have $2j$ separable qubits loaded into my quantum computer, and I want to prepare the permutation symmetric state corresponding to them. How do I do it?! Clearly, there can be no deterministic quantum operation that will do the trick.

<hr>

Well, in this game, what one can't do deterministically, one can often do probablistically. And that turns out to be the case here.

Consider the following circuit:

<img src="img/two_qubit_symmetrizer.png">


The heart of this circuit is the "controlled swap" or Fredkin gate. This is a three qubit gate: if the first qubit is $\uparrow$, it leaves the second two alone; but if the first qubit is $\downarrow$, it swaps/permutes the second two. 

Its matrix representation is:

$$ \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix} $$

Which is easily obtained insofar as its just the $4 \times 4$ identity block concatenated with the usual swap gate:

$$\begin{bmatrix} 1&0&0&0\\0&0&1&0\\0&1&0&0\\0&0&0&1\end{bmatrix}$$

By the way, that's a general rule for constructing controlled gates: just take the gate you want to be controlled by the first qubit, and concatenate it with an identity matrix of the same dimensionality.

So we have our two qubits we want to symmetrize, and the control qubit, with starts in the $\mid \uparrow \rangle$ state. We apply a Hadamard gate to the control, which takes it to an even superposition $\frac{1}{\sqrt{2}}(\mid \uparrow \rangle + \mid \downarrow \rangle$. So then when we apply the Fredkin, the two qubits to be symmetrized end up in *a superposition of being swapped and not being swapped*, relative to the control. We then apply a Hadamard again to the control, which by the way, is its own inverse ($H = H^{\dagger}$), to undo that original rotation, while leaving it entangled with the two qubits.

We then measure the control qubit: if we get $\uparrow$, then our two qubits $\mid \psi \rangle$ and $\mid \phi \rangle$ end up in the symmetrized state $\frac{1}{\sqrt{2}}(\mid \psi \rangle\mid \phi \rangle + \mid \phi \rangle\mid \psi \rangle)$. On the other hand, we get $\downarrow$, then our two qubits ends up in the antisymmetrized state: $\frac{1}{\sqrt{2}}(\mid \psi \rangle\mid \phi \rangle - \mid \phi \rangle\mid \psi \rangle)$.

So if we want the symmetrized state, we just keep doing the experiment over and over again until we get the answer we want! 

In the diagram above, you may note there are also two phase rotation gates. We can use those if we want a symmetrized state that picks up an arbitrary phase when the qubits are swapped. If $\theta=0$, we just get the identity matrix for both of them, and everything is as I said above.

In [3]:
from scipy.linalg import block_diag

# construct our operators
H = (1/np.sqrt(2))*qt.Qobj(np.array([[1,1],\
                                     [1,-1]]))
SWAP = qt.Qobj(np.array([[1,0,0,0],\
                         [0,0,1,0],\
                         [0,1,0,0],\
                         [0,0,0,1]]))
CSWAP = qt.Qobj(block_diag(np.eye(4), SWAP.full()))
CSWAP.dims = [[2,2,2], [2,2,2]]

theta = 0
P1 = qt.Qobj(np.array([[np.exp(1j*theta/2), 0],\
                       [0, 1]]))
P2 = qt.Qobj(np.array([[1, 0],\
                       [0, np.exp(1j*theta/2)]]))

CIRCUIT = qt.tensor(H, qt.identity(2), qt.identity(2))*\
          qt.tensor(P2, qt.identity(2), qt.identity(2))*\
          CSWAP*\
          qt.tensor(P1, qt.identity(2), qt.identity(2))*\
          qt.tensor(H, qt.identity(2), qt.identity(2))

c = qt.basis(2,0) # control
a, b = qt.rand_ket(2), qt.rand_ket(2)
state = qt.tensor(c, a, b)

def measure_control(state):
    Zl, Zv = qt.sigmaz().eigenstates()
    Zp = [v*v.dag() for v in Zv][::-1]
    dm = state.ptrace(0).full().real
    which = np.random.choice(list(range(2)), p=np.diag(dm))
    print("p(up): %.4f" % (dm[0,0]))
    print("p(down): %.4f" % (dm[1,1]))
    print("obtained: %s" % ("up" if which == 0 else "down"))
    return which, (qt.tensor(Zp[which], qt.identity(2), qt.identity(2))*state).unit()

which, state = measure_control(CIRCUIT*state)
answer = state.ptrace((1,2)) # throw out the control

correct_sym = (qt.tensor(a,b) + qt.tensor(b,a)).unit()
correct_sym = correct_sym*correct_sym.dag() # since we need to compare w/ a (pure) density matrix
correct_antisym = (qt.tensor(a,b) - qt.tensor(b,a)).unit()
correct_antisym = correct_antisym*correct_antisym.dag()

if which == 0:
    print("got correct symmetrized state?: %s" % (answer == correct_sym))
else:
    print("got correct antisymmetrized state?: %s" % (answer == correct_antisym))


p(up): 0.7472
p(down): 0.2528
obtained: down
got correct antisymmetrized state?: True


<hr>

Okay, so that's great, but what if we have multiple qubits we want to symmetrize?

The simplest thing to do is a direct generalization of the above. If we have $n$ qubits, there will be $n!$ possible permutations. So we'll need a control qudit that lives in $n!$ dimensions. For example, for $3$ qubits, there are $3! = 6$ possible permutations, so we need a control state that is $6$ dimensional. We start with the control in its $\mid 0 \rangle$ state, and the apply the Hadamard in $n!$ dimensions, in other words, the <a href="https://en.wikipedia.org/wiki/Quantum_Fourier_transform">"Quantum Fourier Transform,"</a> which takes $\mid 0 \rangle$ to an even superposition of all its basis states. We then do a controlled operation, which makes performing the $i^{th}$ permutation on the $n$ qubits depend on the control being in the $\mid i \rangle$ basis state. So we end up with the $n$ qubits being in a superposition of being permuted in all possible ways, relative to the control. Then we apply the inverse QFT to the control, and then measure the control: if it ends up back in the $\mid 0 \rangle$ state, then the $n$ qubits are in the permutation symmetric state. Sweet!

You might wonder how to contruct such a controlled permutation gate. In braket notation, you can think of it like:

$$ \sum_{p_{i} \in perm} \sum_{j=0}^{2^n} \mid i \rangle P_{i}\mid j \rangle \langle i \mid \langle j \mid$$

In other words, we sum over all the permutations $p_{i}$ (where the $i$ labels both a basis state of the control $\mid i \rangle$, and also $P_{i}$, the operator which performs that permutation of the qubits), and also over the $2^n$ basis states of the $n$ qubits, labeled by $j$: $\mid j \rangle$. Clearly, if this operator acts on a state coming in on the right, it will perform the $i^{th}$ permutation to the extent that the control is in the $i^{th}$ state.

Let's check it out:


In [4]:
import math
from itertools import product

# constructs a unitary operation that performs
# the desired permutation of qubit subsystems
def construct_permuter(perm):
    n = len(perm)
    indices = [[(i, j) for j in range(2)] for i in range(n)]
    tensor_indices = list(product(*indices))
    pindices = [[(i, j) for j in range(2)] for i in perm]
    ptensor_indices = list(product(*pindices))
    d = len(tensor_indices)
    m = np.zeros((d,d))
    for i, pind in enumerate(ptensor_indices):
        oi = None
        for p in permutations(pind):
            if p in tensor_indices:
                oi = tensor_indices.index(p)
                break
        m[i, oi] = 1
    m = qt.Qobj(m)
    m.dims = [[2]*n, [2]*n]
    return m

def construct_qft(d):
    w = np.exp(2*np.pi*1j/d)
    return (1/np.sqrt(d))*qt.Qobj(\
                    np.array([[w**(i*j)\
                                   for j in range(d)]\
                                       for i in range(d)]))

# constructs a unitary operator that does each of n!
# permutations of n qubits relative to a control of
# dimensionality n!
def construct_cperms(n):
    ds = 2**n
    dp = math.factorial(n)
    perms = list(permutations(list(range(n))))
    terms = []
    for i, p in enumerate(perms):
        for j in range(ds):
            permuter = construct_permuter(p)
            permuter.dims = [[ds], [ds]]
            terms.append(\
                qt.tensor(qt.basis(dp, i), permuter*qt.basis(ds, j))*\
                    qt.tensor(qt.basis(dp, i), qt.basis(ds, j)).dag())
    O = sum(terms)
    O.dims = [[dp]+[2]*n, [dp]+[2]*n]
    return O

################################################################################

n = 3
dp = math.factorial(n)
qubits = [qt.rand_ket(2) for i in range(n)]

QFT = qt.tensor(construct_qft(dp), *[qt.identity(2)]*n)
CPERMS = construct_cperms(n)
PROJ = qt.tensor(qt.basis(dp, 0)*qt.basis(dp, 0).dag(), *[qt.identity(2)]*n)

state = qt.tensor(qt.basis(dp, 0), *qubits)
state = QFT.dag()*CPERMS*QFT*state

# We're lazy, so let's just project into the right state, regardless of probability
final_state = (PROJ*state).unit().ptrace(list(range(1, n+1))) # throw out the control
correct_state = symmetrize(qubits)
correct_state = correct_state*correct_state.dag() # want to compare with (pure) density matrix

print("got the right permutation symmetric state?: %s" % (final_state == correct_state))

got the right permutation symmetric state?: True


Incidentally, it's worth thinking about the fact that this whole recipe is *not* reversible. Suppose we wanted to go from our symmetrized state of $n$ qubits back to the separable state of $n$ qubits. If we've performed the measurement, and indeed, ended up in the symmetrized state, what happens? We apply the inverse of `QFT.dag()*CPERMS*QFT`: our control ends up in an even superposition as before, and then we condition the permutations on it... but we're in the permutation symmetric state, so it has no effect! 

On the other hand, if we *don't* make that final measurement, then everything is perfectly reversible. Because of linearity, if we apply operations on the qubits, those operations are applied "in each possible world," including the one where they end up totally symmetrized, so a) we can always wait until later to actually measure the control, if we want, and it'll be just as if we measured the control, and then manipulated the symmetrized state; and b) we can apply the above circuit in reverse, and obtain a state of separable qubits whose "stars" have evolved in just the right way. E.g.:

In [5]:
d = n+1
j = (d-1)/2

spin = qt.rand_ket(d) # create a spin
qubits = [qt.Qobj(xyz_spinor(star))\
              for star in spin_XYZ(spin)] # get qubits

S = spin_sym(j)
H = qt.rand_herm(d)#qt.jmat(j, 'x')
U = (-1j*H*np.pi).expm()
spin2 = U*spin # apply random unitary to spin

state = qt.tensor(qt.basis(dp, 0), *qubits)
# apply symmetrization circuit, then upgraded unitary,
# the the inverse of the sym circuit
CIRCUIT = QFT.dag()*CPERMS*QFT
bigH = S*H*S.dag()
bigU = qt.tensor(qt.identity(dp), (-1j*bigH*np.pi).expm())
state = CIRCUIT.dag()*bigU*CIRCUIT*state

final_state = state.ptrace(list(range(1, n+1)))
# convert the final sym state into a spin state
# for comparison
final_spin = (S.dag()*final_state*S).unit()

print("original spin probs:")
print(np.diag((spin*spin.dag()).full()))
print("evolved spin probs:")
print(np.diag((spin2*spin2.dag()).full()))
print("evolved sym probs:")
print(np.diag(final_spin.full()))

original spin probs:
[0.29278106+0.j 0.26911552+0.j 0.20033854+0.j 0.23776488+0.j]
evolved spin probs:
[0.53636766+0.j 0.21956477+0.j 0.06337357+0.j 0.180694  +0.j]
evolved sym probs:
[0.53636765+0.j 0.21956477+0.j 0.06337357+0.j 0.180694  +0.j]


<hr>

Okay, the final concern you might have is that our control system is $n!$ dimensions, where $n$ is the number of qubits we want to symmetrize. We, however, want to formulate *everything* in terms of qubits. Is there a way of using several qubits as controls to get the same effect? Yes! And here's how:

First, let's define:

$$ R_{k} = \frac{1}{\sqrt{k+1}} \begin{bmatrix} 1 & -\sqrt{k} \\ \sqrt{k} & 1 \end{bmatrix}$$

$$ T_{j,j+1} = \frac{1}{\sqrt{k-j+1}} \begin{bmatrix} \sqrt{k-j+1} & 0 & 0 & 0 \\ 0 & 1 & \sqrt{k-j} & 0 \\ 0 & -\sqrt{k-j} & 1 & 0 \\ 0 & 0 & 0 & \sqrt{k-j+1} \end{bmatrix}$$

The point of these guys it to prepare our control qubits. If we have $k$ control qubits in the $\mid 0, 0, 0, \dots \rangle$ state, if we apply $R_{k}$ to the first qubit, and $T_{j, j+1}$'s to adjacent qubits along the line (from $j=1$ to $j = k-1$), then we'll end up in the state:

$$ \frac{1}{\sqrt{k+1}}(\mid 00 \dots 0 \rangle + \mid 10 \dots 0 \rangle + \mid 01 \dots 0 \rangle + \mid 00 \dots 1 \rangle)$$

In other words, a superposition over all the qubits being $\uparrow$ or just one of the qubits being $\downarrow$. (Recall in this notation $\mid 0 \rangle$ = $\mid \uparrow \rangle$, etc.) This is the qubit equivalent of our control qudit from before being prepared in an even superposition of all its basis states. Call this whole procedure $U_{k}$.

So we have $k$ control qubits prepared. Here's the key: this is in the context of having *already symmetrized* $k$ qubits and we want to symmetrize an additional $k+1^{th}$ qubit. Having prepared the $k$ control qubits, we apply $k$ Fredkin gates. If $j$ runs from $1$ to $k$, then the $j^{th}$ Fredkin uses the $j^{th}$ control qubit to condition the swapping of the $j^{th}$ and $k+1^{th}$ qubits. Then, we as before apply the inverse of the preparation of the control qubits, $U_{k}^{\dagger}$.

Okay, so the whole point is to "cascade the above construction" for $k = 1,2,\dots,n-1$, where $n$ is the number of qubits we want to symmetrize. So we start with $k=1$: we've "already symmetrized" one qubit and we want to symmetrize the next one, so we introduce $k=1$ control qubits, preparing them, and then fredkin-ing, and unpreparing. Then we move on to $k=2$, and we add two more control qubits, prepare them, fredkin, and unprepare, etc. In the end, we'll need $\frac{n(n-1)}{2}$ control qubits in total: so if $n=4$, we have $\frac{4(4-1)}{2} = 6$ control qubits. 

Finally, we measure all the controls, and if they're all in the $0/\uparrow$ state, then our qubits will have been symmetrized!

Here's the circuit for the $n=4$ case: 

<img width=400 src="img/four_qubit_symmetrizer.png">

Let's check it out!

In [7]:
from qutip.qip.operations import fredkin

def upgrade(O, i, n):
    return qt.tensor([O if i==j else \
                      qt.identity(2)\
                        for j in range(n)])

def Uk(k):
    return (1/np.sqrt(k+1))*qt.Qobj(np.array([[1, -np.sqrt(k)],\
                                              [np.sqrt(k), 1]]))

def Tj(j, k):
    T = (1/np.sqrt(k-j+1))*qt.Qobj(np.array([[np.sqrt(k-j+1), 0, 0, 0],\
                                             [0, 1, np.sqrt(k-j), 0],\
                                             [0, -np.sqrt(k-j), 1, 0],\
                                             [0, 0, 0, np.sqrt(k-j+1)]]))
    T.dims = [[2,2], [2,2]]
    return T

################################################################################

n = 4
qubits = [qt.rand_ket(2) for i in range(n)]
state = qt.tensor(*qubits)
offset = 0 # keeps track of how many control qubits we've already added
for k in range(1, n):
    controls = qt.tensor(*[qt.basis(2,0)]*k)
    U = upgrade(Uk(k), 0, k)
    for j in range(k-1):
        T = qt.tensor(*[qt.identity(2)]*(j), Tj(j+1, k), *[qt.identity(2)]*(k-j-2))
        U = T*U
    controls = U*controls
    state = qt.tensor(controls, state)
    d = len(state.dims[0])
    for j in range(k):
        state = fredkin(N=d, control=j, targets=[offset+k+j, offset+2*k])*state
    state = qt.tensor(U.dag(), qt.tensor(*[qt.identity(2)]*(d-k)))*state
    offset += k

print("used %d control qubits" % offset)
# do our projections
for i in range(offset):
    state = (upgrade(qt.basis(2,0)*qt.basis(2,0).dag(), i, len(state.dims[0]))*state).unit()

final_state = state.ptrace(list(range(offset, len(state.dims[0]))))
correct_state = symmetrize(qubits)
correct_state = correct_state*correct_state.dag()

print("got the right permutation symmetric state?: %s" % (final_state == correct_state))

used 6 control qubits
got the right permutation symmetric state?: True


## Bibliography

<a href="https://arxiv.org/abs/quant-ph/0212143">Symmetrization and Entanglement of Arbitrary States of Qubits</a>

<a href="refs/Stabilization_of_Quantum_Compu.pdf">Stabilization of Quantum Computations By Symmetrization</a>